In [1]:
import pandas as pd
import plotly.graph_objs as go
import string
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import cufflinks as cf
from plotly.offline import iplot
from tqdm import tqdm

from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
import plotly.express as px

cf.go_offline()
cf.set_config_file(offline = False, world_readable = True)

import plotly.io as pio
pio.renderers.default='notebook'

import warnings
warnings.simplefilter('ignore')

In [2]:
from numpy.linalg import svd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

# 0.IMPORT данных

In [12]:
import os
us = os.getcwd()
us

'C:\\Users\\Владислав\\test_python_scripts\\project\\test project'

In [13]:
if 'liliyaivannikova' in us:
    movies = pd.read_csv(r'/Users/liliyaivannikova/Documents/project/ml-latest/movies.csv')
    rating = pd.read_csv(r'/Users/liliyaivannikova/Documents/project/ml-latest/ratings.csv')
elif 'Владислав' in us:
    movies = pd.read_csv(r'dataset/movies.csv')
    rating = pd.read_csv(r'dataset/ratings.csv')

In [14]:
print(f'''
movies: {movies.shape}
rating: {rating.shape}
''')


movies: (86537, 3)
rating: (33832162, 4)



In [15]:
movies.columns = [col.upper() for col in movies.columns]
movies.head()

,MOVIEID,TITLE,GENRES
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
movies['REALEASE'] = movies['TITLE'].str.extract("\((\d{4})\)", expand=True)
movies['REALEASE'] = pd.to_datetime(movies['REALEASE'], format='%Y')
movies['REALEASE'] = movies['REALEASE'].dt.year
movies['TITLE'] = movies['TITLE'].str[:-7]

In [17]:
movies

,MOVIEID,TITLE,GENRES,REALEASE
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0
...,...,...,...,...
86532,288967,State of Siege: Temple Attack,Action|Drama,2021.0
86533,288971,Ouija Japan,Action|Horror,2021.0
86534,288975,The Men Who Made the Movies: Howard Hawks,Documentary,1973.0
86535,288977,Skinford: Death Sentence,Crime|Thriller,2023.0


In [18]:
dfx = movies['GENRES'].str.get_dummies(sep='|')
for col in dfx.columns:
    dfx[col] = dfx[col].astype('int8')


In [19]:
movies = pd.concat([movies, dfx], axis=1).drop(columns=['GENRES'])
movies['TITLE'] = movies['TITLE'].astype('category')
movies['REALEASE'] = movies['REALEASE'].astype('float16')

In [20]:
movies.dtypes

MOVIEID                  int64
TITLE                 category
REALEASE               float16
(no genres listed)        int8
Action                    int8
Adventure                 int8
Animation                 int8
Children                  int8
Comedy                    int8
Crime                     int8
Documentary               int8
Drama                     int8
Fantasy                   int8
Film-Noir                 int8
Horror                    int8
IMAX                      int8
Musical                   int8
Mystery                   int8
Romance                   int8
Sci-Fi                    int8
Thriller                  int8
War                       int8
Western                   int8
dtype: object

In [21]:
movies.head()

,MOVIEID,TITLE,REALEASE,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995.0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995.0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995.0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,1995.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
rating.columns = [col.upper() for col in rating.columns]
rating.head()

,USERID,MOVIEID,RATING,TIMESTAMP
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [23]:
df = movies.merge(rating, how="left", on="MOVIEID")
df.head()

,MOVIEID,TITLE,REALEASE,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,USERID,RATING,TIMESTAMP
0,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,1.0,4.0,1.225735e+09
1,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,2.0,5.0,8.358160e+08
2,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,7.0,4.0,9.745180e+08
3,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,10.0,3.0,1.430666e+09
4,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,12.0,5.0,8.625007e+08


In [24]:
del rating

# 1. Препроцессинг общей таблицы

In [25]:
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], unit='s')
'''df['YEAR'] = df['TIMESTAMP'].dt.year
df['MONTH'] = df['TIMESTAMP'].dt.month
df['TIME'] = df['TIMESTAMP'].dt.time
df['DATE'] = df['TIMESTAMP'].dt.date
df['WEEK_DAY'] = df['TIMESTAMP'].dt.day_name()
df['DAY_NUM'] = df['TIMESTAMP'].dt.day_of_week'''

"df['YEAR'] = df['TIMESTAMP'].dt.year\ndf['MONTH'] = df['TIMESTAMP'].dt.month\ndf['TIME'] = df['TIMESTAMP'].dt.time\ndf['DATE'] = df['TIMESTAMP'].dt.date\ndf['WEEK_DAY'] = df['TIMESTAMP'].dt.day_name()\ndf['DAY_NUM'] = df['TIMESTAMP'].dt.day_of_week"

In [26]:
#df['RATING'] = df['RATING'].astype('float16')

In [27]:
df.head()

,MOVIEID,TITLE,REALEASE,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,USERID,RATING,TIMESTAMP
0,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,1.0,4.0,2008-11-03 17:52:19
1,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,2.0,5.0,1996-06-26 19:06:11
2,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,7.0,4.0,2000-11-18 03:27:04
3,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,10.0,3.0,2015-05-03 15:19:54
4,1,Toy Story,1995.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,12.0,5.0,1997-05-01 15:32:18


In [28]:
df.dtypes

MOVIEID                        int64
TITLE                       category
REALEASE                     float16
(no genres listed)              int8
Action                          int8
Adventure                       int8
Animation                       int8
Children                        int8
Comedy                          int8
Crime                           int8
Documentary                     int8
Drama                           int8
Fantasy                         int8
Film-Noir                       int8
Horror                          int8
IMAX                            int8
Musical                         int8
Mystery                         int8
Romance                         int8
Sci-Fi                          int8
Thriller                        int8
War                             int8
Western                         int8
USERID                       float64
RATING                       float64
TIMESTAMP             datetime64[ns]
dtype: object

In [29]:
stat_movies = df.groupby('MOVIEID')['USERID'].count().reset_index()

In [30]:
stat_movies

,MOVIEID,USERID
0,1,76813
1,2,30209
2,3,15820
3,4,3028
4,5,15801
...,...,...
86532,288967,1
86533,288971,1
86534,288975,1
86535,288977,1


In [31]:
# зададим как порог минимальной оценки
min_threshold = stat_movies['USERID'].quantile(0.5)
min_threshold

5.0

<font color='red'> **------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------** </font>

<font color='red'> **Убрал записи  пустыми значениями ~3298, это около 0.01%, фигня короче, можно сносить.** </font>

In [43]:
df.shape

(33832162, 26)

In [41]:
df = df[~((df.TIMESTAMP.isna())|(df.USERID.isna())|(df.RATING.isna()))]

In [42]:
train, test = train_test_split(df, test_size=0.3, stratify = df.TIMESTAMP.dt.month)

In [46]:
del df, movies

In [47]:
train.shape

(23682513, 26)

<font color='red'> **------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------** </font>
<font color='red'> **Проба спарс матриц** </font>

!pip install rectools #специальная библиотека

In [51]:
from rectools import Columns
from rectools.dataset import Dataset

In [58]:
df_train = train[['MOVIEID','USERID','RATING','TIMESTAMP']]
df_train.columns = ['user_id','item_id','weight','datetime']
df_train

,user_id,item_id,weight,datetime
22171662,6373,261950.0,4.0,2016-05-12 14:25:15
10442183,1527,135898.0,5.0,2014-10-04 02:45:24
12143004,1983,215432.0,5.0,2007-06-17 00:24:10
22976775,7057,184535.0,3.0,2004-08-08 15:49:38
18380135,3917,119812.0,3.5,2010-03-18 16:01:51
...,...,...,...,...
11527229,1784,270892.0,3.0,2001-10-25 22:57:56
16816671,3248,86786.0,1.0,2017-04-03 10:31:51
7107342,1060,39635.0,3.0,1999-12-25 01:48:26
27035873,56715,113052.0,4.5,2008-11-20 23:46:37


In [59]:
sparse_df_train = Dataset.construct(
    df_train,
    make_dense_user_features=False  # for `sparse` format
)

In [60]:
sparse_df_train

Dataset(user_id_map=IdMap(external_ids=array([  6373,   1527,   1983, ..., 210581, 276561, 143174], dtype=int64)), item_id_map=IdMap(external_ids=array([261950., 135898., 215432., ..., 214184., 269954.,  34221.])), interactions=Interactions(df=          user_id  item_id  weight            datetime
22171662        0        0     4.0 2016-05-12 14:25:15
10442183        1        1     5.0 2014-10-04 02:45:24
12143004        2        2     5.0 2007-06-17 00:24:10
22976775        3        3     3.0 2004-08-08 15:49:38
18380135        4        4     3.5 2010-03-18 16:01:51
...           ...      ...     ...                 ...
11527229      372     3007     3.0 2001-10-25 22:57:56
16816671     3027     6137     1.0 2017-04-03 10:31:51
7107342       173   175371     3.0 1999-12-25 01:48:26
27035873     1056     6647     4.5 2008-11-20 23:46:37
820843        646   155351     4.0 1997-07-23 04:07:18

[23684822 rows x 4 columns]), user_features=None, item_features=None)

In [62]:
sparse_df_train.user_id_map

IdMap(external_ids=array([  6373,   1527,   1983, ..., 210581, 276561, 143174], dtype=int64))

In [63]:
sparse_df_train.item_id_map

IdMap(external_ids=array([261950., 135898., 215432., ..., 214184., 269954.,  34221.]))

In [64]:
sparse_df_train.interactions

Interactions(df=          user_id  item_id  weight            datetime
22171662        0        0     4.0 2016-05-12 14:25:15
10442183        1        1     5.0 2014-10-04 02:45:24
12143004        2        2     5.0 2007-06-17 00:24:10
22976775        3        3     3.0 2004-08-08 15:49:38
18380135        4        4     3.5 2010-03-18 16:01:51
...           ...      ...     ...                 ...
11527229      372     3007     3.0 2001-10-25 22:57:56
16816671     3027     6137     1.0 2017-04-03 10:31:51
7107342       173   175371     3.0 1999-12-25 01:48:26
27035873     1056     6647     4.5 2008-11-20 23:46:37
820843        646   155351     4.0 1997-07-23 04:07:18

[23684822 rows x 4 columns])

> тут я немного не успел до конца разобрать, но вот по ссылке неплохая реализация еще есть + пример как в модель грузить
> https://rekkobook.com/chapter1/collab_filter.html

<font color='red'> **------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------** </font>

In [26]:
df_ratings = train[:10000].pivot(index = 'USERID', columns ='MOVIEID', values = 'RATING').fillna(0).astype('float16')
df_ratings.head()

MOVIEID,1,2,3,5,6,7,9,10,11,12,...,278702,278794,280114,280342,280900,281096,281190,282671,284411,288265
USERID,,,,,,,,,,,,,,,,,,,,,
155.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
224.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df_ratings.dtypes

MOVIEID
1         float16
2         float16
3         float16
4         float16
5         float16
           ...   
281096    float16
281190    float16
284879    float16
285593    float16
286897    float16
Length: 3746, dtype: object

In [ ]:
# TODO
svd = SVD()

start = time.time()

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
svd.fit(trainset)

predictions = svd.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))